In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data\Eta_vs_Theta'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\Eta_vs_Theta'

### ssro calibrations
day = '20160726_' # for ssro calibration
ssro_calib_lt3 = day+'095123'
ssro_calib_lt4 = day+'104600'

Purify_analysis = pa.purify_analysis('Eta_v_Theta',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

msmt_days = ['20160726']
theta_percent = [10,15,25,50,75]

TPQI_vis = 0.5765
TPQI_vis_u = 0.04


In [ ]:
fidelities = []
fidelities_u = []

for theta in theta_percent:
    
    Purify_analysis.get_tstamps_and_offsets(unshifted_days = msmt_days, shifted_days = [], contains = 'eta_'+str(theta)+'percent')
    Purify_analysis.load_raw_data()
    
    ##### Set filters:
    st_start = 3055e3#2763e3
    st_len       = 40e3 #50 ns
    st_len_w2    = 40e3
    
    Purify_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                           st_len = st_len,
                                                           st_len_w2 = st_len_w2
                                                           )
    Purify_analysis.apply_sync_filter_w1_w2()
    corrected_corrs,u_corrected_corrs,no = Purify_analysis.correlate_RO_results_no_purification(apply_ROC = True,return_value = True,verbose=False)

    fidelities.append((corrected_corrs[1]+corrected_corrs[2] + 2*np.sqrt(TPQI_vis*corrected_corrs[1]*corrected_corrs[2]))/2)
    fidelities_u.append(np.sqrt((1+np.sqrt(TPQI_vis*corrected_corrs[2]/corrected_corrs[1]))*u_corrected_corrs[1]**2\
                               +(1+np.sqrt(TPQI_vis*corrected_corrs[1]/corrected_corrs[2]))*u_corrected_corrs[2]**2\
                               +np.sqrt(corrected_corrs[1]*corrected_corrs[2]/TPQI_vis)*TPQI_vis_u**2)/2)


In [ ]:
fig = plt.figure()
ax = plt.subplot()
plt.errorbar(theta_percent,fidelities,fidelities_u)
ax.set_xlabel('Population in ms=0')
ax.set_ylabel('Fidelity')
plt.xlim(np.array(plt.xlim())+[-5,5])
plt.show()
plt.close('all')